<a href="https://colab.research.google.com/github/viniciusrpb/cic0269_natural_language_processing/blob/main/cap04_text_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Instalação de pacotes

In [9]:
!pip install nltk
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importando arquivo csv para o ambiente Colab

In [6]:
!cp -r "/content/drive/MyDrive/edm/prediction/students.csv" "students2.csv"

In [69]:
import pandas as pd
import re
import nltk

In [71]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Criação do objeto dataframe

In [13]:
dataframe = pd.read_csv('students2.csv',sep='|',encoding='utf-8')

In [14]:
dataframe

,Performance,Gender,Caste,coaching,time,Class_ten_education,twelve_education,medium,Class_ X_Percentage,Class_ XII_Percentage,Father_occupation,Mother_occupation
0,Excellent,male,General,NO,ONE,SEBA,AHSEC,ENGLISH,Excellent,Excellent,DOCTOR,OTHERS
1,Excellent,male,OBC,WA,TWO,SEBA,AHSEC,OTHERS,Excellent,Excellent,SCHOOL_TEACHER,HOUSE_WIFE
2,Excellent,male,OBC,OA,TWO,OTHERS,CBSE,ENGLISH,Excellent,Excellent,BUSINESS,HOUSE_WIFE
3,Excellent,male,General,WA,ONE,SEBA,AHSEC,OTHERS,Excellent,Excellent,SCHOOL_TEACHER,SCHOOL_TEACHER
4,Excellent,male,General,OA,TWO,SEBA,CBSE,ENGLISH,Excellent,Excellent,COLLEGE_TEACHER,HOUSE_WIFE
...,...,...,...,...,...,...,...,...,...,...,...,...
661,Average,female,ST,WA,ONE,SEBA,AHSEC,ENGLISH,Good,Vg,OTHERS,HOUSE_WIFE
662,Average,male,ST,WA,THREE,SEBA,AHSEC,ENGLISH,Vg,Good,CULTIVATOR,HOUSE_WIFE
663,Average,male,ST,WA,TWO,SEBA,CBSE,ENGLISH,Good,Vg,OTHERS,SCHOOL_TEACHER
664,Average,male,ST,WA,THREE,SEBA,AHSEC,ENGLISH,Good,Good,SCHOOL_TEACHER,HOUSE_WIFE


In [19]:
df = pd.read_csv('twitter-2013train-A.txt',sep='\t',encoding='utf-8',names=['id','polarity','text'])

In [20]:
df

,id,polarity,text
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going t...
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa an...
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just h...
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome c...
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establi...
...,...,...,...
9679,103158179306807296,positive,RT @MNFootNg It's monday and Monday Night Foot...
9680,103157324096618497,positive,All I know is the road for that Lomardi start ...
9681,100259220338905089,neutral,"All Blue and White fam, we r meeting at Golden..."
9682,104230318525001729,positive,@DariusButler28 Have a great game agaist Tam...


In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/viniciusrpb/cic0269_natural_language_processing/main/corpus_tweets/twitter-2013train-A.txt',sep='\t',encoding='utf-8',names=['id','polarity','text'])

In [24]:
df

,id,polarity,text
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going t...
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa an...
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just h...
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome c...
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establi...
...,...,...,...
9679,103158179306807296,positive,RT @MNFootNg It's monday and Monday Night Foot...
9680,103157324096618497,positive,All I know is the road for that Lomardi start ...
9681,100259220338905089,neutral,"All Blue and White fam, we r meeting at Golden..."
9682,104230318525001729,positive,@DariusButler28 Have a great game agaist Tam...


In [25]:
text_df = df['text'] 

In [26]:
text_df

0       Gas by my house hit $3.39!!!! I\u2019m going t...
1       Theo Walcott is still shit\u002c watch Rafa an...
2       its not that I\u2019m a GSP fan\u002c i just h...
3       Iranian general says Israel\u2019s Iron Dome c...
4       Tehran\u002c Mon Amour: Obama Tried to Establi...
                              ...                        
9679    RT @MNFootNg It's monday and Monday Night Foot...
9680    All I know is the road for that Lomardi start ...
9681    All Blue and White fam, we r meeting at Golden...
9682    @DariusButler28   Have a great game agaist Tam...
9683    I'm pisseeedddd that I missed Kid Cudi's show ...
Name: text, Length: 9684, dtype: object

Para pegar instâncias específicas ou colunas específicas do DataFrame, pode-se utilizar o iloc https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html

In [56]:
sentence = text_df[0]
sentence = sentence+" "+text_df[1]

In [57]:
sentence

'Gas by my house hit $3.39!!!! I\\u2019m going to Chapel Hill on Sat. :) Theo Walcott is still shit\\u002c watch Rafa and Johnny deal with him on Saturday.'

Tokenização: quebrar uma sentença em palavras

In [58]:
sentence = sentence.lower()

In [59]:
sentence

'gas by my house hit $3.39!!!! i\\u2019m going to chapel hill on sat. :) theo walcott is still shit\\u002c watch rafa and johnny deal with him on saturday.'

In [63]:
tokens = re.split(r'[\s!$@#,.;]+',sentence)

In [62]:
tokens

['gas',
 'by',
 'my',
 'house',
 'hit',
 '3.39',
 'i\\u2019m',
 'going',
 'to',
 'chapel',
 'hill',
 'on',
 'sat.',
 ':)',
 'theo',
 'walcott',
 'is',
 'still',
 'shit\\u002c',
 'watch',
 'rafa',
 'and',
 'johnny',
 'deal',
 'with',
 'him',
 'on',
 'saturday.']

Remoção de stopwords

In [64]:
stop_list = ['and','or','the','is','on','in']

In [65]:
novel_tokens = []
for t in tokens:
    if t not in stop_list:
        novel_tokens.append(t)

In [66]:
novel_tokens

['gas',
 'by',
 'my',
 'house',
 'hit',
 '3',
 '39',
 'i\\u2019m',
 'going',
 'to',
 'chapel',
 'hill',
 'sat',
 ':)',
 'theo',
 'walcott',
 'still',
 'shit\\u002c',
 'watch',
 'rafa',
 'johnny',
 'deal',
 'with',
 'him',
 'saturday',
 '']

In [74]:
stop_list_nltk = nltk.corpus.stopwords.words('english')

In [75]:
stop_list_nltk

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [77]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [78]:
tokens_final = [stemmer.stem(t) for t in novel_tokens]

In [79]:
tokens_final

['ga',
 'by',
 'my',
 'hous',
 'hit',
 '3',
 '39',
 'i\\u2019m',
 'go',
 'to',
 'chapel',
 'hill',
 'sat',
 ':)',
 'theo',
 'walcott',
 'still',
 'shit\\u002c',
 'watch',
 'rafa',
 'johnni',
 'deal',
 'with',
 'him',
 'saturday',
 '']

In [80]:
novel_tokens

['gas',
 'by',
 'my',
 'house',
 'hit',
 '3',
 '39',
 'i\\u2019m',
 'going',
 'to',
 'chapel',
 'hill',
 'sat',
 ':)',
 'theo',
 'walcott',
 'still',
 'shit\\u002c',
 'watch',
 'rafa',
 'johnny',
 'deal',
 'with',
 'him',
 'saturday',
 '']